In [1]:
import pandas as pd
import numpy as np
import sys 
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import plotly.express as px
import itertools
import optuna

import math

from collections import defaultdict
import re

from joblib import Parallel, delayed
sys.path.insert(0, '../DevCode')
pd.set_option('display.expand_frame_repr', False)
pd.options.display.max_rows = 500

In [2]:
# from DataManager import SimpleTokenizer

In [3]:
import torch
import torch.utils.data as utils
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

import torch.optim as optim


from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [4]:
from collections import defaultdict
import re
import pandas as pd

class SimpleTokenizer:
    def __init__(self):
        self.token_to_id = defaultdict(lambda: len(self.token_to_id))
        self.id_to_token = {}
        self.token_to_id['<PAD>'] = 0  # Padding token

    def tokenize(self, text):
        # Simple tokenization by splitting on non-word characters
        tokens = re.findall(r'\w+|\S', text)
        return tokens

    def convert_tokens_to_ids(self, tokens):
        ids = []
        for token in tokens:
            token_id = self.token_to_id[token]
            self.id_to_token[token_id] = token  # Store the reverse mapping
            ids.append(token_id)
        return ids

    def decode(self, token_ids):
        # Convert a list of token IDs back to a string
        tokens = [self.id_to_token.get(token_id, '') for token_id in token_ids]
        return ''.join(tokens)  # Modify this line as needed for your URL format

    def tokenize_column(self, series):
        # Apply tokenization to each row in the pandas series
        tokenized = series.apply(self.tokenize)
        # Convert each list of tokens to a list of token IDs
        tokenized_ids = tokenized.apply(self.convert_tokens_to_ids)
        return tokenized_ids

    def run_convert(self):
        pass


In [5]:
df = pd.read_csv('./Data/malicious_phish.csv')
tm = SimpleTokenizer()

In [6]:
tokenized_url = tm.tokenize_column(df['url'])

In [7]:
df['tokens'] = tokenized_url

In [8]:
df = df.rename({'type':'target'},axis=1)
df = df.drop(['url',],axis=1)
df['sizes'] = [len(i) for i in df['tokens']]
mean = df['sizes'].mean()
std = df['sizes'].std()
clipped_data = df[(df['sizes'] >= (mean - 2*std)) & (df['sizes'] <= (mean + 2*std))]
data = df.loc[clipped_data.index]
targets = df['target'].loc[clipped_data.index]

max_length = 0

for i in data['tokens'].values:
    if len(i) > max_length:
        max_length = len(i)
array_data = [np.array(i) for i in data['tokens'].values]

matrix_list = []
for i in array_data:
    new_vector = np.zeros(max_length)
    current_vector_size = i.shape[0]
    new_vector[:current_vector_size] = i
    matrix_list.append(new_vector)

matrix_main = np.vstack(matrix_list)

In [9]:
matrix_main.shape

(629456, 47)

In [10]:
# from DataManager import SimpleTokenizer
# df = pd.read_csv('./Data/malicious_phish.csv')
# tm = SimpleTokenizer()
# df['string_tokens'] = tm.tokenize_column(df['url'])
# df['tokens'] = df['string_tokens'].apply(tm.convert_tokens_to_ids)
# df = df.rename({'type':'target'},axis=1)
# df = df.drop(['url', 'string_tokens'],axis=1)
# df['sizes'] = [len(i) for i in df['tokens']]
# mean = df['sizes'].mean()
# std = df['sizes'].std()
# clipped_data = df[(df['sizes'] >= (mean - 2*std)) & (df['sizes'] <= (mean + 2*std))]
# data = df.loc[clipped_data.index]
# targets = df['target'].loc[clipped_data.index]

# max_length = 0

# for i in data['tokens'].values:
#     if len(i) > max_length:
#         max_length = len(i)
# array_data = [np.array(i) for i in data['tokens'].values]

# matrix_list = []
# for i in array_data:
#     new_vector = np.zeros(max_length)
#     current_vector_size = i.shape[0]
#     new_vector[:current_vector_size] = i
#     matrix_list.append(new_vector)

# matrix_main = np.vstack(matrix_list)

In [11]:
matrix_main.max()

627328.0

In [12]:
data

,target,tokens,sizes
0,phishing,"[1, 2, 3, 4, 5, 4, 1]",7
1,benign,"[6, 4, 5, 7, 8, 7, 9, 4, 10]",9
2,benign,"[11, 4, 12, 7, 13, 7, 14, 7, 15, 4, 16]",11
3,defacement,"[17, 18, 7, 7, 19, 4, 20, 2, 21, 4, 22, 7, 23,...",31
4,defacement,"[17, 18, 7, 7, 36, 2, 37, 4, 38, 7, 23, 4, 24,...",25
...,...,...,...
651186,phishing,"[62048, 4, 3247, 4, 5, 7, 3248, 7, 3758, 7, 62...",13
651187,phishing,"[2686, 4, 45660, 4, 5, 7, 23449, 2, 4843, 7, 1...",16
651188,phishing,"[19, 4, 8613, 4, 5, 7, 62048, 7, 5920, 7, 4298...",12
651189,phishing,"[187, 4, 188, 4, 12, 7, 189, 7, 627327, 327, 3...",12


In [38]:
matrix_main

array([[1.0000e+00, 2.0000e+00, 3.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [6.0000e+00, 4.0000e+00, 5.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [1.1000e+01, 4.0000e+00, 1.2000e+01, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       ...,
       [1.9000e+01, 4.0000e+00, 8.6130e+03, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [1.8700e+02, 4.0000e+00, 1.8800e+02, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [1.9000e+01, 4.0000e+00, 1.5712e+04, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00]])

In [10]:
fish = np.array(data[data['target'] == 'phishing'].index)

In [11]:
data_matrix = matrix_main

In [12]:
fish_matrix = []
count = 0
it = 0
for i in fish[:-3]:
    try:
        fish_matrix.append(data_matrix[i])
        count += 1
        it = i
    except:
        continue

In [13]:
fish_matrix = np.vstack(fish_matrix)

In [13]:
fish_matrix.shape

(70789, 47)

In [14]:
fish_matrix.max()

627328.0

In [10]:
data_matrix.shape[0]

629456

In [6]:
data_matrix = data_matrix[:10000]

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TransformerModel(nn.Module):
    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__()
        from torch.nn import TransformerEncoder, TransformerEncoderLayer
        self.model_type = 'Transformer'
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        self.decoder = nn.Linear(ninp, ntoken)

        self.init_weights()

    def generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src):
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            device = src.device
            mask = self.generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask

        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, self.src_mask)
        output = self.decoder(output)
        return output

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)


In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
device

device(type='cuda')

In [16]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

class URLDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        # Assuming the first token is the start token, and we want to predict the next token
        # Input sequence (excluding the last token)
        input_seq = item[:-1]
        # Target sequence (excluding the first token)
        target_seq = item[1:]
        return torch.tensor(input_seq, dtype=torch.long), torch.tensor(target_seq, dtype=torch.long)


In [17]:
# url_dataset = URLDataset(data_matrix)
url_dataset = URLDataset(fish_matrix)
url_dataloader = DataLoader(url_dataset, batch_size=10, shuffle=True)

In [26]:
import time

In [27]:
count = 0
for batch, (input_seq, target_seq) in enumerate(url_dataloader):
    input_seq = input_seq.to(device)
    target_seq = target_seq.to(device)

    count += 1

    time.sleep(0.5)

    input_seq = input_seq.cpu()
    target_seq = target_seq.cpu()

    if count >= 10:
        break

    


In [57]:
input_seq = input_seq.cpu()

In [58]:
target_seq = target_seq.cpu()

In [20]:
torch.cuda.empty_cache()

In [17]:
# ntokens = int(matrix_main.max() + 1)  # size of vocabulary; replace 'token_vocab' with your vocabulary size
ntokens = int(fish_matrix.max() + 1)


emsize = 32  # embedding dimension
nhid = 32  # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2  # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2  # the number of heads in the multiheadattention models
dropout = 0.1  # the dropout value

model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

/home/john/localCyberProject2/.venv/lib/python3.10/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [103]:
5e-5

5e-05

In [105]:
5e-10

5e-10

In [18]:
epochs = 50

for epoch in range(1, epochs + 1):
    model.train()
    total_loss = 0.
    epoch_loss = 0.0
    for batch, (input_seq, target_seq) in enumerate(url_dataloader):
        input_seq = input_seq.to(device)
        target_seq = target_seq.to(device)

        optimizer.zero_grad()
        output = model(input_seq)
        loss = criterion(output.view(-1, ntokens), target_seq.view(-1))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()


        total_loss += loss.item()
        epoch_loss = total_loss / len(url_dataloader)
        input_seq = input_seq.cpu()
        target_seq = target_seq.cpu()
    
    print(f'Epoch {epoch}, Loss: {total_loss}')
    torch.save(model.state_dict(), 'model.pt')
    torch.save(optimizer.state_dict(), 'optimizer.pt')

Epoch 1, Loss: 10149.232485830784
Epoch 2, Loss: 8143.904314637184
Epoch 3, Loss: 7879.199747681618
Epoch 4, Loss: 7790.785867631435
Epoch 5, Loss: 7749.44854837656
Epoch 6, Loss: 7726.926665574312
Epoch 7, Loss: 7712.372817873955
Epoch 8, Loss: 7701.594541430473
Epoch 9, Loss: 7691.053180336952
Epoch 10, Loss: 7682.116796135902
Epoch 11, Loss: 7672.123938918114
Epoch 12, Loss: 7664.866341531277
Epoch 13, Loss: 7657.399979740381
Epoch 14, Loss: 7650.173413097858
Epoch 15, Loss: 7646.674731492996
Epoch 16, Loss: 7641.134645760059
Epoch 17, Loss: 7635.3813281953335
Epoch 18, Loss: 7625.385862708092
Epoch 19, Loss: 7621.964706689119
Epoch 20, Loss: 7612.969972491264
Epoch 21, Loss: 7613.102441251278
Epoch 22, Loss: 7606.136807799339
Epoch 23, Loss: 7604.728419959545
Epoch 24, Loss: 7598.545760810375
Epoch 25, Loss: 7598.571486890316
Epoch 26, Loss: 7598.292203843594
Epoch 27, Loss: 7595.529361963272
Epoch 28, Loss: 7591.979817211628
Epoch 29, Loss: 7597.497672021389
Epoch 30, Loss: 7597.3

In [ ]:
Epoch 15, Loss: 7860.097127854824

In [28]:
# ntokens = int(matrix_main.max() + 1)  # size of vocabulary; replace 'token_vocab' with your vocabulary size
ntokens = int(fish_matrix.max() + 1)


emsize = 32  # embedding dimension
nhid = 32  # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2  # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2  # the number of heads in the multiheadattention models
dropout = 0.1  # the dropout value

model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

/home/john/localCyberProject2/.venv/lib/python3.10/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [29]:
model.load_state_dict(torch.load('model.pt'))

<All keys matched successfully>

In [30]:
optimizer.load_state_dict(torch.load('optimizer.pt'))

In [24]:
model = torch.load('model.pt')

In [27]:
type(model)

torch.nn.modules.module._IncompatibleKeys

In [36]:
def generate_url(start_token, max_len=20):
    model.eval()
    with torch.no_grad():
        input_seq = torch.tensor([start_token], dtype=torch.long).unsqueeze(1).to(device)
        for _ in range(max_len - 1):
            output = model(input_seq)
            next_token = torch.argmax(output[-1], dim=-1).item()
            next_token_tensor = torch.tensor([next_token], dtype=torch.long).unsqueeze(1).to(device)
            input_seq = torch.cat([input_seq, next_token_tensor], dim=0)
            
            if next_token == 0:  # Replace with your end-of-sequence token
                break
        return input_seq.squeeze().cpu().numpy()

# Example usage
generated_url_tokens = generate_url(start_token=3)  # Replace '1' with your start token


In [32]:
generated_url_tokens

array([3, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7])

In [34]:
from collections import defaultdict

In [50]:
tk = SimpleTokenizer()

In [23]:
torch.save(model.state_dict(), 'model.pt')

In [21]:
url = generated_url_tokens = generate_url(start_token=2)
url

array([     2, 521410,      4,      5,      7,      0])

In [100]:
url = generated_url_tokens = generate_url(start_token=17)
decoded = tm.decode(url)

In [37]:
for i in range(1,200):
    url = generated_url_tokens = generate_url(start_token=i)
    decoded = tm.decode(url)
    print(url)
    print(decoded)
    print('')

[1 7 0]
br/

[     2 521410      4      5      7      7      7      7      7      7
      7      7      7      7      7      7      7      7      7      7]
-jyuku.com////////////////

[3 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7]
icloud///////////////////

[4 5 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7]
.com//////////////////

[5 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7]
com///////////////////

[6 4 5 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7]
mp3raid.com/////////////////

[ 7  7  7  7  7  7 19  4  5  7  7  7  7  7  7  7  7  7  7  7]
//////www.com///////////

[8 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7]
music///////////////////

[  9   2 476   2 476   2 476   2 476   2 476   2 476   2 476   2 476   2
 476   2]
krizz_kaliko-to-to-to-to-to-to-to-to-to-

[10  0]
html

[11  4 10  0]
bopsecrets.html

[12  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7]
org///////////////////

[    13      2 521410      4      5      7      7      7      7      0]
rexroth-jyuku.com////

[    14      2 521410      4      5 

In [101]:
url

array([17, 18,  7,  7,  7,  0])

In [99]:
decoded

'http:///'

In [17]:
# Model parameters
ntokens = int(matrix_main.max() + 1)  # size of vocabulary; replace 'token_vocab' with your vocabulary size
emsize = 32  # embedding dimension
nhid = 32  # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2  # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2  # the number of heads in the multiheadattention models
dropout = 0.2  # the dropout value

model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

def train(dataloader):
    model.train()
    total_loss = 0.
    for batch, (input_seq, target_seq) in enumerate(dataloader):
        input_seq = input_seq.to(device)
        target_seq = target_seq.to(device)

        optimizer.zero_grad()
        output = model(input_seq)
        loss = criterion(output.view(-1, ntokens), target_seq.view(-1))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
    return total_loss / len(dataloader)

# Number of epochs
epochs = 10  # You can adjust this

for epoch in range(1, epochs + 1):
    epoch_loss = train(url_dataloader)
    print(f'Epoch {epoch}, Loss: {epoch_loss}')


/home/john/localCyberProject2/.venv/lib/python3.10/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


KeyboardInterrupt: 